# OpenAI API 가이드

- OpenAI의 기존 **Chat Completions API**와 2025년 3월 공개된 차세대 **Responses API**를 모두 포함
- Chat Completions vs Responses API 비교

    | 특징 | Chat Completions API | Responses API |
    |------|----------------------|---------------|
    | **상태 관리** | Stateless (클라이언트가 이력 관리) | **Stateful** (서버가 ID로 관리) |
    | **도구 사용** | Function Calling (구현 복잡) | **Built-in Tools** (웹/파일/코드 내장) |
    | **주 용도** | 단순 텍스트 생성, 챗봇 | **복합 에이전트**, 워크플로우 자동화 |


---

## Part 1: Chat Completions API

### 1.1 LLM(Large Language Model)의 생성 원리

LLM은 대규모 텍스트 데이터를 학습하여 자연어를 이해하고 생성하는 모델입니다.
- **트랜스포머 구조**를 기반으로 하며, 문맥을 파악하여 **다음 단어를 예측**하는 방식으로 작동합니다.
- **토큰(Token)** 단위로 텍스트를 처리하며, 확률 분포에 따라 가장 적절한 토큰을 선택하여 문장을 완성합니다.

    <div style="text-align: left; font-size: 12px;">
    <div style="text-align: center; background-color: white;">
        <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/34/Transformer%2C_full_architecture.png/440px-Transformer%2C_full_architecture.png"
            alt="Illustrations for the Transformer and attention mechanism showing the full Transformer architecture"
            width="600"
            style="border: 0;">
    </div>

    **Image Title:** Transformer Architecture Illustration  
    **Source:** [GitHub - DL Visuals](https://github.com/dvgodoy/dl-visuals/?tab=readme-ov-file)  
    **License:** [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/)  
    **Author(s):** dvgodoy  

    </div>



### 1.2 Chat Completions API 개요

OpenAI의 가장 대표적인 API로, 대화형 인터페이스를 통해 텍스트를 생성합니다.
- **Messages 구조**: 시스템(System), 사용자(User), 보조자(Assistant) 역할을 가진 메시지 리스트를 입력으로 받습니다.
- **Stateless**: 기본적으로 상태를 저장하지 않으므로, 대화의 맥락을 유지하려면 이전 대화 내용을 매번 함께 전송해야 합니다.

### 1.3 환경 설정 및 패키지 설치

먼저 필요한 패키지를 설치하고 환경 변수를 설정합니다.


In [ ]:
# 패키지 설치
# !pip install langchain langchain-openai python-dotenv ipykernel
# !uv add langchain langchain-openai python-dotenv ipykernel

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# .env 파일에서 API 키 로드
load_dotenv()

# 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
os.getenv("OPENAI_API_KEY")

### 1.4 기본 사용법: 텍스트 생성

가장 기본적인 형태의 대화 요청입니다.


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini", 
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "파이썬에서 리스트 컴프리헨션에 대해 설명해줘."}
    ],
    temperature=0.7,
    max_tokens=500
)

print(response.choices[0].message.content)

In [ ]:
# 결과 출력 (Markdown)
from IPython.display import Markdown, display

display(Markdown(response.choices[0].message.content))

### 1.5 텍스트 요약 예제

긴 텍스트를 입력받아 요약하는 함수를 만들어봅니다.


In [ ]:
def summarize_text(text, model="gpt-4o-mini"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful summarization assistant."},
            {"role": "user", "content": f"다음 텍스트를 요약해주세요:\n\n{text}"}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content

sample_text = """
인공지능(AI)은 인간의 학습, 추론, 지각, 문제 해결 능력 등 지적 능력을 컴퓨터로 구현하는 기술이다. 
오늘날 인공지능은 다양한 분야에서 활용되고 있으며, 특히 패턴 인식, 자연어 처리, 기계 학습 등의 
분야에서 큰 발전을 이루었다. 최근에는 딥러닝 기술의 발전으로 이미지 인식, 음성 인식, 번역 등의 
작업에서 인간과 비슷하거나 더 뛰어난 성능을 보이는 사례가 늘고 있다.
"""

print(summarize_text(sample_text))


---

## Part 2: Responses API

- **Responses API**는 2025년 3월 공개된 OpenAI의 새로운 API
- 기존 Chat Completions API의 기능을 통합하고 확장한 **"에이전트용 통합 API"**

### 2.1 왜 Responses API인가?

- 기존 Chat Completions API는 강력하지만 몇 가지 한계가 있었음
    - **Stateless**: 매 요청마다 전체 대화 이력을 다시 보내야 해서 토큰 비용이 낭비됩니다.
    - **도구 통합의 복잡성**: 검색이나 코드 실행 같은 도구를 연결하려면 클라이언트 측에서 복잡한 로직을 구현해야 합니다.

- **Responses API의 주요 이점:**
    1.  ✅ **서버 측 상태 관리 (Stateful)**: `previous_response_id`만 보내면 이전 대화 맥락이 유지됩니다.
    2.  ✅ **네이티브 도구 통합**: 웹 검색, 파일 분석, 코드 실행 기능이 API 내부에 내장되어 있습니다.
    3.  ✅ **멀티모달 지원**: 텍스트와 이미지를 자연스럽게 섞어서 입력할 수 있습니다.


### 2.2 기본 사용법: 단순 텍스트 응답

클라이언트 사용법은 기존과 유사하지만, `client.responses.create` 메서드를 사용합니다.


In [ ]:
# Responses API 호출
response = client.responses.create(
    model="gpt-4o-mini",
    input="Responses API의 가장 큰 장점은 무엇인가요?"
)

# 응답 출력 (구조가 약간 다릅니다)
print(response.output[0].content[0].text)


---
## Part 3: Responses API 주요 기능

### 3.1 상태 유지 (Stateful) 대화

이전 대화 내용을 클라이언트가 관리할 필요 없이, ID만으로 맥락을 이어갑니다.


In [ ]:
# 첫 번째 요청
response1 = client.responses.create(
    model="gpt-4o-mini",
    input="내 이름은 김철수야. 기억해줘."
)
print(f"Response 1: {response1.output[0].content[0].text}")

In [ ]:
# ID 저장
response_id = response1.id

# 두 번째 요청 (이전 응답 ID 참조)
response2 = client.responses.create(
    model="gpt-4o-mini",
    input="내 이름이 뭐였지?",
    previous_response_id=response_id  # 핵심: 이전 상태 참조
)
print(f"Response 2: {response2.output[0].content[0].text}")

### 3.2 멀티모달 입력 (텍스트 + 이미지)

이미지 파일을 base64로 인코딩하여 텍스트와 함께 전송할 수 있습니다.


In [ ]:
import base64

def encode_image(image_path: str) -> str:
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

image_data = encode_image("data/celltrion_report_chart.jpg")

response = client.responses.create(
    model="gpt-4o-mini",  
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",            # ✅ 텍스트는 input_text
                    "text": "이 차트의 주요 트렌드를 설명해줘."
                },
                {
                    "type": "input_image",           # ✅ 이미지는 input_image
                    "image_url": f"data:image/jpeg;base64,{image_data}"
                }
            ],
        }
    ],
)

# 응답 텍스트 출력
print(response.output[0].content[0].text)

### 3.3 내장 도구: 웹 검색 (Web Search)

별도의 구현 없이 `tools` 파라미터에 설정을 추가하는 것만으로 웹 검색이 가능합니다.


In [ ]:
response = client.responses.create(
    model="gpt-4o-mini",    # gpt-4o-mini 모델은 미지원
    tools=[
        {
            "type": "web_search"
        }
    ],
    input="2024년 노벨 물리학상 수상자는 누구인가요? 최신 정보를 찾아주세요."
)

response.model_dump()


In [ ]:
print(response.output[1].content[0].text)

### 3.4 구조화된 출력 (JSON Output)

응답을 특정 JSON 스키마에 맞춰 출력하도록 강제할 수 있습니다. 데이터 추출 작업에 유용합니다.


In [ ]:
import json

response = client.chat.completions.create(
    model="gpt-4o-mini", 
    messages=[
        {"role": "user", "content": "다음 문장에서 인물과 도시를 추출해줘: '스티브 잡스는 샌프란시스코에서 자랐다.'"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "person_city",
            "schema": {
                "type": "object",
                "properties": {
                    "person": {"type": "string"},
                    "city": {"type": "string"}
                },
                "required": ["person", "city"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

print(response.choices[0].message.content)